In [2]:
import pandas as pd
import numpy as np

In [1]:
### File Paths

pri_path = '../data/predicted/pri_comp_inv.csv'
sec_path = '../data/predicted/sec_comp_inv.csv'
tri_path = '../data/predicted/tri_comp_inv.csv'

In [3]:
def read_and_clean_csv(file_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Drop rows with NaN values
    df_cleaned = df.dropna(how='all')
    
    return df_cleaned

In [4]:
# Read and clean the CSV files
pri_df = read_and_clean_csv(pri_path)
sec_df = read_and_clean_csv(sec_path)
tri_df = read_and_clean_csv(tri_path)

---

In [5]:
def update_inventory_values(data):
    supply_columns = ['B01_supply', 'B02_supply', 'B03_supply', 'S01_supply', 'S02_supply', 'S03_supply']
    inv_columns = ['B01_inv', 'B02_inv', 'B03_inv', 'S01_inv', 'S02_inv', 'S03_inv']
    
    for supply_col, inv_col in zip(supply_columns, inv_columns):
        data[inv_col] = data.apply(lambda row: row[inv_col] if row[supply_col] > 0 else 0, axis=1)
    
    return data

In [6]:
pri_df_cln = update_inventory_values(pri_df)
sec_df_cln = update_inventory_values(sec_df)
tri_df_cln = update_inventory_values(tri_df)

---


In [9]:
supply_columns = ['B01_supply', 'B02_supply', 'B03_supply', 'S01_supply', 'S02_supply', 'S03_supply']
inv_columns = ['B01_inv', 'B02_inv', 'B03_inv', 'S01_inv', 'S02_inv', 'S03_inv']


In [11]:
comparison_results = pd.DataFrame(columns=['Supply_Column', 'Inv_Comparison'])


In [12]:
for supply_col, inv_col in zip(supply_columns, inv_columns):
    # Get the _inv values from each DataFrame
    pri_inv = pri_df_cln[inv_col]
    sec_inv = sec_df_cln[inv_col]
    tri_inv = tri_df_cln[inv_col]
    
    # Calculate differences or ratios
    diff_pri_sec = (pri_inv - sec_inv).abs()  # Absolute difference between pri and sec
    ratio_pri_sec = (pri_inv / sec_inv).fillna(0)  # Ratio of pri to sec, handle division by zero with fillna
    diff_pri_tri = (pri_inv - tri_inv).abs()  # Absolute difference between pri and tri
    ratio_pri_tri = (pri_inv / tri_inv).fillna(0)  # Ratio of pri to tri, handle division by zero with fillna
    
    # Store comparison results in the DataFrame
    comparison_results = comparison_results.append({'Supply_Column': supply_col, 'Inv_Comparison': f"PRI-Sec: {diff_pri_sec.mean()}, PRI/Tri: {ratio_pri_tri.mean()}, PRI-Tri: {diff_pri_tri.mean()}, PRI/Tri: {ratio_pri_tri.mean()}"}, ignore_index=True)


C:\Users\MOTHERSHIP\AppData\Local\Temp\ipykernel_52060\560067714.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comparison_results = comparison_results.append({'Supply_Column': supply_col, 'Inv_Comparison': f"PRI-Sec: {diff_pri_sec.mean()}, PRI/Tri: {ratio_pri_tri.mean()}, PRI-Tri: {diff_pri_tri.mean()}, PRI/Tri: {ratio_pri_tri.mean()}"}, ignore_index=True)
C:\Users\MOTHERSHIP\AppData\Local\Temp\ipykernel_52060\560067714.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comparison_results = comparison_results.append({'Supply_Column': supply_col, 'Inv_Comparison': f"PRI-Sec: {diff_pri_sec.mean()}, PRI/Tri: {ratio_pri_tri.mean()}, PRI-Tri: {diff_pri_tri.mean()}, PRI/Tri: {ratio_pri_tri.mean()}"}, ignore_index=True)
C:\Users\MOTHERSHIP\AppData\Local\Temp\ipykernel_52060\560067714.py:14: FutureWarning: The

In [14]:
comparison_results

,Supply_Column,Inv_Comparison
0,B01_supply,"PRI-Sec: 2765.4612483459496, PRI/Tri: 0.574712..."
1,B02_supply,"PRI-Sec: 1867.0063684449221, PRI/Tri: 0.886334..."
2,B03_supply,"PRI-Sec: 17.52136370292334, PRI/Tri: 0.0383141..."
3,S01_supply,"PRI-Sec: 3.448683448009814, PRI/Tri: 0.0025542..."
4,S02_supply,"PRI-Sec: 31.4625630553858, PRI/Tri: 0.04980842..."
5,S03_supply,"PRI-Sec: 12.222106753467012, PRI/Tri: 0.019157..."
